# Day 12 Challenge

In [58]:
import aocd

session:str = "53616c7465645f5f5d64e0f6b2811f4d18eb862dae5aa906e4d25c0f6a1d5944c89433699f1b1175fdea140e981da4a5"
day = 18
year = 2019
data = aocd.get_data(session=session, year=year, day=day)

In [120]:
import networkx as nx
import numpy as np
from matplotlib import pyplot as plt
from numpy import min


def plot_graph(obj):
        
    """Plots the simplified graph object."""

    plt.figure(figsize=(20,20))

    nx.draw_kamada_kawai(
        obj,

        # node options
        with_labels=True,
        node_size=1200,
        node_color="black",
        font_family="consolas",
        font_weight='bold',
        font_color = "w",
        font_size=24,
    )
    plt.show()

    
class Navigator:
    
    """Class allows navigation in a labyrinth object."""
    
    def __init__(self, data):
        
        self.alphabet = ""
        for char in data:
            if char not in " .#\n":
                self.alphabet += char
                
        self.key_range = ""
        for char in self.alphabet:
            if char == str.lower(char):
                if char != "@":
                    self.key_range += char
                

        # convert map from input data
        self._map = data.strip().split("\n")
        self._amap = None
        
        self._navigator_position = None
        
        # Locate all items in map incl. own position
        self._map_items = {"doors": {}, "keys": {}}
        
        # Initialize all internal objects
        self._extract_map_items()
        self._str_map_to_array()
        
        # prepare graph object
        self._g = None
        self._simple_g = None
        
        # create graph
        self._construct_graph()
        self._construct_simple_graph()
        
        # walk preparations
        self._keys = []
        
        # player's current position and traveling distance
        self._cc = "@"
        self._d = 0
        
        print("""Welcome to NAVIGATOR!


 _______              .__              __                
 \\      \\ _____ ___  _|__| _________ _/  |_  ___________ 
 /   |   \\\\__  \\\\  \\/ /  |/ ___\\__  \\\\   __\\/  _ \\_  __ \\
/    |    \\/ __ \\\\   /|  / /_/  > __ \\|  | (  <_> )  | \\/
\\____|__  (____  /\\_/ |__\\___  (____  /__|  \\____/|__|   
        \\/     \\/       /_____/     \\/                   


You can use the _simple_graph object to see neighboring keys
and doors, or you can use the _g object to explor the complete graph.""")
    
    # Private methods
    
    def _str_map_to_array(self):
        
        """Turns a string map into an numpy array map."""
        
        out_map = []
        for row in self._map:
            out_map.append(list(row))
        self._amap = np.array(out_map)
    
    def _extract_map_items(self):
        
        """Extracts all non-wall map items from given map."""
        
        r = 0
        for row in self._map:
            c = 0
            for char in row:
                if char not in "#.":
                    if char == "@":
                        self._navigator_position = (c, r)
                    elif char != "@":
                        if char == str.upper(char):
                            self._map_items["doors"][char] = (c, r)
                        elif char == str.lower(char):
                            self._map_items["keys"][char] = (c, r)
                c += 1
            r += 1
        self._map_items
    
    def _construct_graph(self):
        xmax, ymax = self._amap.shape

        nodes = []
        edges = []

        # walk the inner area of the mace (1 to -1)
        for i in range(1,xmax-1):
            for j in range(1,ymax-1):

                # read current character
                cc = self._amap[i,j]

                # if not a wall look around to finde paths
                if cc != "#":
                    if cc == ".":
                        cc = f"V-{i}-{j}"
                    if cc not in nodes:
                        nodes.append(cc)

                    # look what's above, right, below, left
                    rc = self._amap[i+1,j]         
                    if rc != "#":
                        if rc == ".":
                            rc = f"V-{i+1}-{j}"
                        if rc not in nodes:
                            nodes.append(rc)

                        edges.append((cc,rc))

                    bc = self._amap[i,j+1]
                    if bc != "#":
                        if bc == ".":
                            bc = f"V-{i}-{j+1}"
                        if bc not in nodes:
                            nodes.append(rc)

                        edges.append((cc,bc))

        self._g = nx.Graph()
        self._g.add_nodes_from(nodes)
        self._g.add_edges_from(edges)
        
    def _construct_simple_graph(self):
        
        """Constructs a simplified graph that only contains doors and keys."""
        
        #alphabet = "@abcdefghijklmnopqrstuvwxyzABZDEFGHIJKLMNOPQRSTUVWXYZ"
        alphabet = ""
        for char in data:
            if char not in " .#\n":
                alphabet += char

        nodes = []
        edges = []

        for fchar in alphabet:
            if fchar not in nodes:
                    nodes.append(fchar)
            for tchar in alphabet:
                if tchar not in nodes:
                    nodes.append(tchar)

                # path between nodes
                path = self._return_shortest_clean_path(fchar, tchar)
                for i in range(len(path)-1):
                    edges.append(
                        (
                            path[i],
                            path[i+1]
                        )
                    )

        self._simple_g = nx.Graph()
        self._simple_g.add_nodes_from(nodes)
        self._simple_g.add_edges_from(edges)
        
    
    def _return_shortest_clean_path(self, start, end):
        
        """Returns cleaned shortest path with no intermediate hops"""
        
        path = self.return_shortest_path(start, end)
        out_path = []
        for element in path:
            if len(element) < 2:
                out_path.append(element)
        return out_path
    
    def _return_neighbors(self, position=None):
        
        """Returns all neighbor elements of a"""
        
        neighbors = []
        distances = []
        
        if not position:
            position = self._cc
        
        for neighbor in self._simple_g.neighbors(position):
            distances.append(nx.shortest_path_length(self._g, self._cc, neighbor))
            neighbors.append(neighbor)
        
        return list(neighbors), distances
    
    # Public methods
    
    def return_shortest_path(self, start, end):
        
        """Returns the shortest path steps as list."""
        
        return nx.shortest_path(self._g, start, end)
        
    
    def return_shortest_path_len(self, start, end):
        
        """Returns the shortest path length."""
        
        return nx.shortest_path_length(self._g, start, end)

    
    def return_neighboring_open_keys(self):
    
        alphabet = ""
        for char in data:
            if char not in " .#\n":
                alphabet += char
    
        destinations = list(nav._map_items["keys"].keys())
        cc_bag_of_paths = []
        cc_destinations = []
        cc_distances = []

        bag_of_keys = []

        for destination in [x for x in destinations if x not in self._keys]:
            cc_bag_of_paths.append(nx.shortest_path(self._simple_g, self._cc, destination))

        for path in cc_bag_of_paths:
            for element in path:

                # skip first one
                if element == self._cc:
                    continue
                
                # skip the ultra starting position
                if element == "@":
                    continue
                    
                # skip keys that have already been collected
                if element in self._keys:
                    continue

                # if element is an unlocked door
                if element == str.upper(element):
                    if str.lower(element) in self._keys:
                        continue
                    else:
                        break

                if element == str.lower(element) and element not in self._keys:
                    if element not in cc_destinations:
                        cc_destinations.append(element)
                    break

        for destination in cc_destinations:
            cc_distances.append(nx.shortest_path_length(self._g, nav._cc, destination))
        
        return cc_destinations, cc_distances
        
    
    def return_nearest_open_key(self):
        
        """Returns the open key which is closest to where the player is."""
        
        n, d = nav.return_neighboring_open_keys()
        return n[d.index(min(d))], min(d)
    
    
    def return_any_open_key(self):
        
        """Returns the open key which is closest to where the player is."""
        
        n, d = nav.return_neighboring_open_keys()
        
        r = random.randint(0, len(n)-1)
        
        return n[r], d[r]
            
    
    def walk_maze(self):
        
        """Collect all keys by taking the shortest possible paths."""

        while True:

            # break if you found all keys
            if len(self._keys) == len(self._map_items["keys"]):
                print(f"Alright, after walking {self._d} steps, I have all my keys.")
                break

            n, d = self.return_nearest_open_key()
            
            print(f"Go to {n}, which takes me {d} steps. Already walked {self._d}")
            
            self._cc = n
            self._keys.append(n)
            self._d += d
            
        return self._d
    
    def random_walk_maze(self):
        
        """Collect all keys by taking the shortest possible paths."""

        while True:

            # break if you found all keys
            if len(self._keys) == len(self._map_items["keys"]):
                break

            n, d = self.return_any_open_key()
            
            self._cc = n
            self._keys.append(n)
            self._d += d
            
        return self._d
    
    def reset(self):
        self._keys = []
        self._d = 0
        self._cc = "@"

## Testing

In [37]:
data = """#########
#b.A.@.a#
#########"""

In [44]:
data = """########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################"""

In [51]:
data = """#################
#i.G..c...e..H.p#
########.########
#j.A..b...f..D.o#
########@########
#k.E..a...g..B.n#
########.########
#l.F..d...h..C.m#
#################"""

In [54]:
data = """########################
#@..............ac.GI.b#
###d#e#f################
###A#B#C################
###g#h#i################
########################"""

In [121]:
nav = Navigator(data=data)

Welcome to NAVIGATOR!


 _______              .__              __                
 \      \ _____ ___  _|__| _________ _/  |_  ___________ 
 /   |   \\__  \\  \/ /  |/ ___\__  \\   __\/  _ \_  __ \
/    |    \/ __ \\   /|  / /_/  > __ \|  | (  <_> )  | \/
\____|__  (____  /\_/ |__\___  (____  /__|  \____/|__|   
        \/     \/       /_____/     \/                   


You can use the _simple_graph object to see neighboring keys
and doors, or you can use the _g object to explor the complete graph.


In [123]:
nav.walk_maze()

Go to r, which takes me 16 steps. Already walked 0
Go to l, which takes me 40 steps. Already walked 16
Go to b, which takes me 14 steps. Already walked 56
Go to p, which takes me 96 steps. Already walked 70
Go to q, which takes me 30 steps. Already walked 166
Go to f, which takes me 16 steps. Already walked 196
Go to x, which takes me 26 steps. Already walked 212
Go to t, which takes me 180 steps. Already walked 238
Go to h, which takes me 54 steps. Already walked 418
Go to g, which takes me 264 steps. Already walked 472
Go to k, which takes me 290 steps. Already walked 736
Go to m, which takes me 324 steps. Already walked 1026
Go to e, which takes me 66 steps. Already walked 1350
Go to j, which takes me 286 steps. Already walked 1416
Go to u, which takes me 14 steps. Already walked 1702
Go to d, which takes me 416 steps. Already walked 1716
Go to s, which takes me 460 steps. Already walked 2132
Go to c, which takes me 492 steps. Already walked 2592
Go to y, which takes me 10 steps. Al

4498

## Further steps

Make a simulator that does not select by `min` but tries every element of the destination list (`cc_destinations`). Calculate the whole thing trough and store the result in an output list. Then, finally, take the smallest element from that list.

Test this approach with: 

```python
data = """########################
#f.D.E.e.C.b.A.@.a.B.c.#
######################.#
#d.....................#
########################"""
```

**Do not repeat the same shit over and over again:** Already submitted 2725, 2724 (original). Corrected: 4498

In [131]:
def parseInput(input=data):
    maze = {}
    goals = {}
    for y,row in enumerate(input.split('\n')):
        for x,cell in enumerate(row):
            p = complex(x,y)
            maze[p] = cell
            if cell in '#.': continue
            goals[cell] = p
    return maze, goals

def findLinks(maze, start):
    links = {}
    walk = defaultdict(lambda:[99999,{}])
    walk[start] = (0,set())
    next = [(start,set())]

    for step in count(1):
        if len(next)==0: break
        cur,next = next,[]
        for p,ds in cur:
            for d in [1,1j,-1,-1j]:
                c = maze[p+d]
                if c == '#' or walk[p+d][0]<=step: continue
                if c.islower():
                    links[c] = (step,ds)
                nds = ds
                if c.isupper():
                    nds = nds | {c.lower()}
                walk[p+d] = (step,nds)
                next.append((p+d,nds))
    return links # naturally sorted by distance

In [137]:
def findShortest1():
    maze, goals = parseInput(input=data)

    allKeys = {k for k in goals if k.islower()}
    links = {'@': findLinks(maze, goals['@'])}
    for k in allKeys:
        links[k] = findLinks(maze, goals[k])

    cache = {}
    def walk(name, needKeys):
        if len(needKeys)==0:
            return 0

        key = name + ''.join(needKeys)
        if key in cache:
            return cache[key]

        shortest = float('inf')
        for k in needKeys:
            l,doors = links[name][k]
            if l >= shortest: continue # too long to try
            if not doors.isdisjoint(needKeys): continue # can't open doors
            tail = walk(k, needKeys - {k})
            if shortest > l + tail: shortest = l + tail
        cache[key] = shortest
        return shortest
    
    res = walk('@', allKeys)
    print('cached',len(cache))
    return res

In [139]:
from collections import defaultdict
from itertools import count

findShortest1()

cached 111072


3856